## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.random.seed(11)

## Load Data

In [ ]:
data = pd.read_excel("/Total_exp.xlsx")
data = data.dropna()
data = data[["Parameter","Freq"]]

FileNotFoundError: ignored

In [ ]:
data.head()

## Storing stack values & frequency

In [ ]:
X_raw = [s.split("_") for s in list(data["Parameter"])]
Y_raw = list(data["Freq"])

## Map stack to a value for plotting purposes

In [ ]:
def transform(X):
    X = X.reshape(-1,6)
    vals = []
    for i in range(X.shape[0]):
      val = 0.0
      for j,x in enumerate(list(X[i])[::-1]):
          val += 4**j * (x/50)
      vals.append([val])
    return np.array(vals)/1000

## Store all available samples

In [ ]:
bounds = np.array([[0, 150],
                   [0, 150],
                   [0, 150],
                   [0, 150],
                   [0, 150],
                   [0, 150]])

X = np.array(X_raw,dtype="float")
Y = np.array(Y_raw,dtype="float")

## Initial datapoints

In [ ]:
n_sample = 2
idxs = np.random.randint(0,X.shape[0],n_sample)

X_init = np.array([X[i] for i in idxs])
Y_init = np.array([[Y[i]] for i in idxs])

In [ ]:
from scipy.stats import norm

def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.

    Args:
        X: Points at which EI shall be computed (m x d).
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.
        xi: Exploitation-exploration trade-off parameter.

    Returns:
        Expected improvements at points X.
    '''
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)

    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    # See also section 2.4 in [...]
    mu_sample_opt = np.max(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0

    return ei

In [ ]:
from scipy.optimize import minimize

def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    '''
    Proposes the next sampling point by optimizing the acquisition function.

    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.

    Returns:
        Location of the acquisition function maximum.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None

    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)

    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x

    return min_x.reshape(-1, 1)

In [ ]:
def get_closest_stack(X,Y,X_estimate):
    idx = -1
    min_dist = 1e20
    for i in range(X.shape[0]):
        dist = np.sum(np.abs(X_estimate-X[i]))
        if dist < min_dist :
            min_dist = dist
            idx = i
    return X[idx], Y[idx]

In [ ]:
def plot_approximation(gpr, X, X_sample, Y_sample, X_next):
    mu, std = gpr.predict(X, return_std=True)
    X = transform(X).ravel()
    idxs = np.argsort(X)
    X = X[idxs]
    Y = mu[idxs].reshape(-1)
    std = std[idxs]
    plt.fill_between(X, Y + 1.96 * std, Y - 1.96 * std, alpha=0.25)
    plt.plot(X, Y, 'b-', lw=1, label='Surrogate function')
    plt.plot(transform(X_sample).ravel(), Y_sample, 'kx', mew=3, label='Noisy samples')
    plt.axvline(x=transform(X_next).ravel(), ls='--', c='k', lw=1)
    plt.legend()

def plot_acquisition(X, Y, X_next):
    X = transform(X).ravel()
    idxs = np.argsort(X)
    X = X[idxs]
    Y = Y[idxs]
    plt.plot(X, Y, 'r-', lw=1, label='Acquisition function')
    plt.axvline(x=transform(X_next).ravel(), ls='--', c='k', lw=1, label='Next sampling location')
    plt.legend()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern

# Gaussian process with Matérn kernel as surrogate model
# Use custom kernel and estimator to match previous example
noise = 0.1
m52 = ConstantKernel(1.0) * Matern(length_scale=[20.0,20.0,20.0,20.0,20.0,20.0], nu=2.5)
gpr = GaussianProcessRegressor(kernel=m52, alpha=noise**2, n_restarts_optimizer=2)

# Initialize samples
X_sample = X_init
Y_sample = Y_init
X_grid = X

# Number of iterations
n_iter = 10

max_frequency = []
stack_ids = []

plt.figure(figsize=(12, n_iter * 3))
plt.subplots_adjust(hspace=0.4)

for i in range(n_iter):
    # Update Gaussian process with existing samples
    gpr.fit(X_sample, Y_sample)

    # Obtain next sampling point from the acquisition function (expected_improvement)
    X_estimate = propose_location(expected_improvement, X_sample, Y_sample, gpr, bounds)
    X_next, Y_next = get_closest_stack(X,Y,X_estimate)

    mu = gpr.predict(X_grid).reshape(-1)

    # Plot samples, surrogate function, noise-free objective and next sampling location
    plt.subplot(n_iter, 2, 2 * i + 1)
    plot_approximation(gpr, X_grid, X_sample, Y_sample, X_next)
    plt.title(f'Iteration {i+1}')

    plt.subplot(n_iter, 2, 2 * i + 2)
    plot_acquisition(X_grid, expected_improvement(X_grid, X_sample, Y_sample, gpr), X_next)

    max_frequency.append(float(np.max(mu)))
    stack_ids.append(np.argmax(mu))

    # Add sample to previous samples
    X_sample = np.vstack((X_sample, X_next))
    Y_sample = np.vstack((Y_sample, Y_next))

## Max frequency & corresponding stack value

In [ ]:
for i in range(n_iter):
    print("Iteration {}: Max frequency -> {}, stack_ value -> {}".format(i+1,max_frequency[i],X[stack_ids[i]]))